In [ ]:
import sys
sys.path.append("..")

In [ ]:
from collections import namedtuple
from enum import Enum

from resources.breadth_first_solver.breadth_first_solver import BreadthFirstSolver
from resources.breadth_first_solver.game import Game
from resources.utils import get_puzzle_input

### Part 1

In [ ]:
class Direction(Enum):
    UP = 0
    LEFT = 1
    RIGHT = 2
    DOWN = 3
    
ordered_directions = (
    Direction.UP,
    Direction.LEFT,
    Direction.RIGHT,
    Direction.DOWN,
)
    
MOVE_MAP = {
    Direction.LEFT: (-1, 0),
    Direction.RIGHT: (1, 0),
    Direction.UP: (0, -1),
    Direction.DOWN: (0, 1),
}

def direction_move(pos, direction):
    offset_x, offset_y = MOVE_MAP[direction]
    return pos[0] + offset_x, pos[1] + offset_y

In [ ]:
MoveState=namedtuple('MoveState', [
    'moved',
    'attacked',
    'killed',
    'can_continue',
    'deadlock',
])

In [ ]:
class PlayerMover(Game):
    """Breadth first search for shortest path to opponent"""
    def __init__(self, player):
        self.player = player
        self.grid = player.grid
        super().__init__()

    @property
    def initial_state(self):
        return self.player.position
    
    def next_moves(self, state):
        return tuple(
            d
            for d in ordered_directions
            if self.grid.free_space(direction_move(state, d))
        )

    def next_state(self, state, move):
        """Next state of the system given move `move` in state `state`."""
        return direction_move(state, move)

    def is_end_state(self, state):
        """Is the current state `state` an end state of the game."""
        return self.player.is_attack_mode(if_at_position=state)

In [ ]:
def prompt_manual_input(player):
    """When it gets too complex they're quite easy to eyeball!"""
    grid = player.grid
    for y in range(grid.max_y + 1):
        line =[]
        for x in range(grid.max_x + 1):
            char = '.'
            if (x, y) in grid.walls:
                char = '#'
            else:
                other_player = grid.player_at_position((x, y))
                if other_player:
                    if other_player == player:
                        char = 'e' if other_player.is_elf else 'g'
                    else:
                        char = 'E' if other_player.is_elf else 'G'
                else:
                    char = '.'
            line.append(char)
        print(''.join(line))

    while True:
        which_way = input("Cannot move {}. Which way?".format(player))
        input_map = {
            'U': (Direction.UP, False),
            'D': (Direction.DOWN, False),
            'L': (Direction.LEFT, False),
            'R': (Direction.RIGHT, False),
            'S': (None, False),
            'X': (None, True)
        }
        if which_way in input_map:
            return input_map[which_way]    

In [ ]:
class Player:
    def __init__(self, id, x, y, is_elf, attack_power, grid):
        self.id = id
        self.x = x
        self.y = y
        self.is_elf = is_elf
        self.attack_power = attack_power
        self.hit_points = 200
        self.grid = grid
        
    @property
    def position(self):
        return self.x, self.y
    
    @property
    def dead(self):
        return self.hit_points <= 0
    
    @property
    def read_position(self):
        return self.y, self.x

    @property
    def opponent_positions(self):
        return {player.position for player in self.grid.opponents(self)}

    def neighbouring_opponents(self, if_at_position=None):
        neighbours = []
        opponent_positions = self.opponent_positions
        pos = if_at_position or self.position

        for d in ordered_directions:
            neighbour_pos = direction_move(pos, d)
            if neighbour_pos in opponent_positions:
                neighbours.append(self.grid.player_at_position(neighbour_pos))
              
        return neighbours
        
    def is_attack_mode(self, if_at_position=None):
        return len(self.neighbouring_opponents(if_at_position)) > 0
        
    def move(self, deadlock=False):
        # If there are no more opponents the game is over
        if not self.opponent_positions:
            return MoveState(
                moved=False,
                attacked=False,
                killed=False,
                can_continue=False,
                deadlock=True
            )
        
        if self.is_attack_mode():
            did_kill = self.attack()
            return MoveState(
                moved=False,
                attacked=True,
                killed=did_kill,
                can_continue=True,
                deadlock=False,
            )
        
        # So we're not next to an opponent. We move towards the closest one to engage.
        # Move in the sortest path to the opponent.
        if deadlock:
            # Nothing can move until a battle resolves
            return MoveState(
                moved=False,
                attacked=False,
                killed=False,
                can_continue=True,
                deadlock=False
            )

        solver = BreadthFirstSolver()
        mover = PlayerMover(self)
        
        direction = None
        deadlock = False
        try:
            paths = solver.solve(mover, max_moves=25)
            paths = sorted(paths, key=lambda p: p[0].value)
            if paths:
                direction = paths[0][0]
        except BufferError:
            direction, deadlock = prompt_manual_input(self)
        
        if direction:
            print('{} moves {}'.format(self, direction))
            self.x, self.y = direction_move(self.position, direction)
            
        # May now be able to attack...
        attacked = False
        has_killed = False
        if self.is_attack_mode():
            attacked = True
            has_killed = self.attack()
            
        return MoveState(
            moved=bool(direction),
            attacked=attacked,
            killed=has_killed,
            can_continue=True,
            deadlock=deadlock
        )

    def hit(self, player):
        print('{} attacks {}'.format(self, player))
        player.hit_points -= self.attack_power
            
    def attack(self):
        weakest_opponent = None
        for opponent in self.neighbouring_opponents():
            weakest_opponent = self.weakest(weakest_opponent, opponent)

        self.hit(weakest_opponent)
        return weakest_opponent.dead
    
    @staticmethod
    def weakest(player_1, player_2):
        if not player_1:
            return player_2
        
        if not player_2:
            return player_1
        
        if player_1.hit_points < player_2.hit_points:
            return player_1
        
        if (
            player_1.hit_points == player_2.hit_points and
            player_1.read_position < player_2.read_position
        ):
            return player_1
        
        return player_2

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(self, type(other)) and self.id == other.id

    def __lt__(self, other):
        return self.y < other.y or (self.y == other.y and self.x < other.x)
    
    def __repr__(self):
        character = 'E' if self.is_elf else 'G'
        return "<{}{}: pos:({},{}), hit_points:{}, attack_power:{}>".format(
            character,
            self.id,
            self.x,
            self.y,
            self.hit_points,
            self.attack_power
        )

In [ ]:
class Arena:
    def __init__(
        self,
        lines,
        elf_power=3,
        goblin_power=3,
        stop_on_elf_down=False
    ):
        self.walls = set()
        self.max_x = 0
        self.max_y = 0
        self.players = set()
        self.game_over = False
        self.completed_rounds = 0
        self.prev_goblin_activity = True
        self.prev_elf_activity = True
        self.elf_power=elf_power
        self.goblin_power=goblin_power
        self.stop_on_elf_down = stop_on_elf_down
        
        self._parse_lines(lines)
        
    def move(self, prev_activity=True):
        print()
        print('Round: ', self.completed_rounds + 1)
        print('Previous Activity: elf={}, goblin={}'.format(
            self.prev_elf_activity,
            self.prev_goblin_activity,
        ))

        round_goblin_activity = False
        round_elf_activity = False
        explicit_deadlock = False

        for player in sorted(self.players):
            if player.dead:
                continue
            
            #print('Moving {}...'.format(player))

            if player.is_elf:
                deadlock = not self.prev_elf_activity and not round_elf_activity
            else:
                deadlock = not self.prev_goblin_activity and not round_goblin_activity            
            deadlock = explicit_deadlock or deadlock
                
            move_state = player.move(deadlock)
            
            # Check for end states
            if not move_state.can_continue:
                print("{} can't move".format(player))
                self.game_over = True
                break
            
            if (
                self.stop_on_elf_down and
                move_state.killed and
                not player.is_elf
            ):
                print("ELF DOWN, ELF DOWN!!!!")
                self.game_over = True
                break
            
            if move_state.moved:
                if player.is_elf:
                    round_elf_activity = True
                else:
                    round_goblin_activity = True

            if move_state.killed:
                explicit_deadlock = False
                round_elf_activity = True
                round_goblin_activity = True
                
            if move_state.deadlock:
                explicit_deadlock = True   
        
        if not self.game_over:
            self.completed_rounds += 1
            
        self.prev_goblin_activity = round_goblin_activity
        self.prev_elf_activity = round_elf_activity
            
    def survivors(self):
        return {player for player in self.players if not player.dead}
        
    def opponents(self, player):
        return [
            other_player
            for other_player in self.players
            if other_player.is_elf != player.is_elf and not other_player.dead
        ]
    
    def free_space(self, position):
        return position not in self.walls and not self.player_at_position(position)

    def player_at_position(self, position):
        for player in self.players:
            if player.position == position and not player.dead:
                return player
        return None

    def _parse_lines(self, lines):
        y = 0
        for line in lines:
            if y > self.max_y:
                self.max_y = y
            x = 0
            for char in line:
                if x > self.max_x:
                    self.max_x = x
                if char == '#':
                    self.walls.add((x, y))
                elif char in ('E', 'G'):
                    is_elf = char == 'E'
                    attack_power = self.elf_power if is_elf else self.goblin_power
                    
                    player = Player(
                        id=len(self.players),
                        x=x,
                        y=y,
                        is_elf=is_elf,
                        attack_power= attack_power,
                        grid=self
                    )
                    self.players.add(player)
                x += 1
            y += 1
    
    def __repr__(self):
        lines = []
        for y in range(self.max_y + 1):
            line =[]
            for x in range(self.max_x + 1):
                char = '.'
                if (x, y) in self.walls:
                    char = '#'
                else:
                    player = self.player_at_position((x, y))
                    if player:
                        char = 'E' if player.is_elf else 'G'
                    else:
                        char = '.'
                line.append(char)
            lines.append(''.join(line))

        return '\n'.join(lines)
   

In [ ]:
test_input = """#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....# 
#######
""".split('\n')

In [ ]:
test_arena = Arena(test_input)
test_arena

In [ ]:
test_arena = Arena(test_input)
while not test_arena.game_over:
    test_arena.move()

```
Before the 48th round can finish, the top-left Goblin finds that there are no targets remaining, and so combat ends. So, the number of full rounds that were completed is 47, and the sum of the hit points of all remaining units is 200+131+59+200 = 590. From these, the outcome of the battle is 47 * 590 = 27730.
```

In [ ]:
test_arena.completed_rounds * sum([player.hit_points for player in test_arena.survivors()])

In [ ]:
puzzle_input = get_puzzle_input('/tmp/day_15.txt')

In [ ]:
len(puzzle_input[0])

In [ ]:
puzzle_arena = Arena(puzzle_input)
print(puzzle_arena)

In [ ]:
puzzle_arena = Arena(puzzle_input)

while not puzzle_arena.game_over:
    puzzle_arena.move()

In [ ]:
puzzle_arena.completed_rounds

In [ ]:
sum([player.hit_points for player in puzzle_arena.survivors()])

In [ ]:
90 * 2403

### Part 2

```
According to your calculations, the Elves are going to lose badly. Surely, you won't mess up the timeline too much if you give them just a little advanced technology, right?

You need to make sure the Elves not only win, but also suffer no losses: even the death of a single Elf is unacceptable.

However, you can't go too far: larger changes will be more likely to permanently alter spacetime.

So, you need to find the outcome of the battle in which the Elves have the lowest integer attack power (at least 4) that allows them to win without a single death. The Goblins always have an attack power of 3.

In the first summarized example above, the lowest attack power the Elves need to win without losses is 15:

#######       #######
#.G...#       #..E..#   E(158)
#...EG#       #...E.#   E(14)
#.#.#G#  -->  #.#.#.#
#..G#E#       #...#.#
#.....#       #.....#
#######       #######

Combat ends after 29 full rounds
Elves win with 172 total hit points left
Outcome: 29 * 172 = 4988
In the second example above, the Elves need only 4 attack power:

#######       #######
#E..EG#       #.E.E.#   E(200), E(23)
#.#G.E#       #.#E..#   E(200)
#E.##E#  -->  #E.##E#   E(125), E(200)
#G..#.#       #.E.#.#   E(200)
#..E#.#       #...#.#
#######       #######

Combat ends after 33 full rounds
Elves win with 948 total hit points left
Outcome: 33 * 948 = 31284
In the third example above, the Elves need 15 attack power:

#######       #######
#E.G#.#       #.E.#.#   E(8)
#.#G..#       #.#E..#   E(86)
#G.#.G#  -->  #..#..#
#G..#.#       #...#.#
#...E.#       #.....#
#######       #######

Combat ends after 37 full rounds
Elves win with 94 total hit points left
Outcome: 37 * 94 = 3478
In the fourth example above, the Elves need 12 attack power:

#######       #######
#.E...#       #...E.#   E(14)
#.#..G#       #.#..E#   E(152)
#.###.#  -->  #.###.#
#E#G#G#       #.#.#.#
#...#G#       #...#.#
#######       #######

Combat ends after 39 full rounds
Elves win with 166 total hit points left
Outcome: 39 * 166 = 6474
In the last example above, the lone Elf needs 34 attack power:

#########       #########   
#G......#       #.......#   
#.E.#...#       #.E.#...#   E(38)
#..##..G#       #..##...#   
#...##..#  -->  #...##..#   
#...#...#       #...#...#   
#.G...G.#       #.......#   
#.....G.#       #.......#   
#########       #########   

Combat ends after 30 full rounds
Elves win with 38 total hit points left
Outcome: 30 * 38 = 1140
After increasing the Elves' attack power until it is just barely enough for them to win without any Elves dying, what is the outcome of the combat described in your puzzle input?
```

In [ ]:
part_two_arena = Arena(puzzle_input, elf_power=15, stop_on_elf_down=True)

while not part_two_arena.game_over:
    if part_two_arena.completed_rounds >= 50:
        print(part_two_arena)
    part_two_arena.move()

In [ ]:
part_two_arena.survivors()

In [ ]:
part_two_arena.completed_rounds

In [ ]:
part_two_arena.completed_rounds * sum([player.hit_points for player in part_two_arena.survivors()])

```
{<E23: pos:(19,4), hit_points:164, attack_power:20>,
 <E19: pos:(20,4), hit_points:164, attack_power:20>,
 <E20: pos:(22,5), hit_points:146, attack_power:20>,
 <E16: pos:(24,5), hit_points:128, attack_power:20>,
 <E17: pos:(18,6), hit_points:95, attack_power:20>,
 <E12: pos:(23,6), hit_points:35, attack_power:20>,
 <E26: pos:(18,7), hit_points:200, attack_power:20>,
 <E29: pos:(19,8), hit_points:200, attack_power:20>,
 <E28: pos:(11,18), hit_points:140, attack_power:20>,
 <E25: pos:(12,21), hit_points:161, attack_power:20>}
 ```